In [1]:
import os
import csv
import pandas as pd
from datetime import datetime
import requests
from bs4 import BeautifulSoup

from sqlalchemy import create_engine
from sqlalchemy.inspection import inspect
from sqlalchemy import create_engine, MetaData, Table, select

from collections import defaultdict

# The following code does the following
1. Takes all speeches and sorts it by President
2. Writes the speeches into individual CSVs 

In [2]:
def scrape(url):
    content = requests.get(url)
    return BeautifulSoup(content.text, 'html.parser')

In [3]:
def write_csv(df, new_path):
    df.to_csv(new_path,encoding='utf-8-sig',index=True)

# Maps dict of list of presidents to id_number

In [4]:
all_presidents_url = 'https://www.presidency.ucsb.edu/presidents'
soup = scrape(all_presidents_url)
soup.prettify().encode('utf8')
texts = soup.findAll(text=True)
start_index = texts.index('Donald J. Trump')
end_index = texts.index('George Washington')
texts = texts[start_index: end_index+1]
dict_all_prezs = dict()
count = 45
for elem in texts:
    if not elem.isdigit() and elem != ' to ' and elem != ' ' and elem != '\n':
        dict_all_prezs[elem] = count
        count -= 1
# dict_all_prezs

# SQL on Speeches/Presidents

In [5]:
engine = create_engine('sqlite://', echo=False)
path = '/Users/Andey/Desktop/jupyter/all_presidents_cleaned.csv'
df = pd.read_csv(path)
df.to_sql('speeches', con=engine)
conn = engine.connect()
metadata = MetaData(conn)
s = Table('speeches', metadata, autoload=True)
s_col = [m.key for m in s.columns]
s_col

['index',
 'speech_id',
 'title',
 'date',
 'year',
 'president',
 'president_id',
 'content',
 'url',
 'footnote',
 'speech_type']

# Sorts and Writes CSV by Presidents

In [6]:
#sorts CSV
lst_by_prez = []
for prez in dict_all_prezs:
    innerLst = (engine.execute("SELECT * FROM speeches WHERE president = '"+ prez +"'").fetchall())
    lst_by_prez.append([prez, innerLst])

In [7]:
#writes CSV
col_names = ['bad_idx','speech_id','title','date','year','president','president_id','content','url','footnote','speech_type']
csv_prez_folder = '/Users/Andey/Desktop/Fall2020/CIRP_Lab/speech_by_president/'
# print(lst_by_prez[0])
for i in lst_by_prez:
    prez = i[0]
    speech_info = i[1]
    try: #write into csv files
        df = pd.DataFrame(speech_info, columns=col_names)
        keep_col = ['speech_id','title','date','year','president','president_id','content','url','footnote','speech_type']
        new_path = csv_prez_folder+prez+'.csv'
        df = df[keep_col]
        df.index.name = 'index'
#         write_csv(df, new_path)
        
    except ValueError: #catches any errors
        print(prez)

# Normalizes speeches
- sorts by president chronologically
- renames index column to 'index' for simplicity sake

In [8]:
speech_info = engine.execute("SELECT * FROM speeches ORDER BY president_id desc").fetchall()
df = pd.DataFrame(speech_info, columns=col_names)
keep_col = ['speech_id','title','date','year','president','president_id','content','url','footnote','speech_type']
new_path = '/Users/Andey/Desktop/Fall2020/CIRP_Lab/speech_by_president/all_sorted_by_prez.csv'
df = df[keep_col]
df.index.name = 'index'
# write_csv(df, new_path)

# Parsing Country Codes from Kountry_ado.txt file
- converted the ".ado file" to a ".txt file"
- parsed it into an array of countries called "all_countries"

In [9]:
#used to parse the kountry_ado file that has other names of countries
def get_inside_quotes(s):
    word_lst = []
    word = ""
    first_quote = False
    i = 0
    while i < (len(s)): 
        if s[i] == '"':
            i += 1
            first_quote = not first_quote
            word_lst.append(word)
            word = ""
        if first_quote:
            word += s[i]
        i += 1
        
    for w in word_lst:
        if w == '':
            word_lst.remove(w)
    return word_lst

In [10]:
#parses each line and puts it into a 2D list of different names
kountry_ado_file = "/Users/Andey/Desktop/jupyter/kountry_ado.txt"
ado = open(kountry_ado_file,"r+") 
all_countries = []
for i in ado:
    all_countries.append(get_inside_quotes(i[20:]))
all_countries

[['Afghanistan', 'afghanistan', 'afghan', 'afganistan'],
 ['Albania', 'albania'],
 ['Algeria', 'algeria'],
 ['American Samoa', 'american samoa', 'eastern samoa', 'samoa american'],
 ['Andorra', 'andorra'],
 ['Angola', 'angola'],
 ['Anguilla', 'anguilla'],
 ['Antigua and Barbuda',
  'antigua and barbuda',
  'antigua barbuda',
  'antigua barbuda'],
 ['Argentina', 'argentina', 'argent'],
 ['Armenia', 'armenia'],
 ['Aruba', 'aruba'],
 ['Australia', 'australia', 'austral'],
 ['Austria', 'austria'],
 ['Austria-Hungary', 'austriahungary'],
 ['Azerbaijan', 'azerbaijan'],
 ['Azores', 'azores'],
 ['Baden', 'baden'],
 ['Bahamas', 'bahamas'],
 ['Bahrain', 'bahrain'],
 ['Bangladesh', 'bangladesh', 'bngldsh'],
 ['Barbados', 'barbados', 'barbado'],
 ['Bavaria', 'bavaria'],
 ['Belarus', 'belarus', 'byelorussia', 'belarus byelorussia'],
 ['Belgium', 'belgium'],
 ['Belgium-Luxembourg', 'belgiumluxembourg', 'bel_lux'],
 ['Belize', 'belize'],
 ['Benin', 'benin', 'dahomey', 'benin/dahomey', 'benin dahomey'

# Normalizing COW Country Codes
The following:
1. Drops duplicates
2. Adds different nicknames for each country into the Country Codes

In [11]:
country_codes = "/Users/Andey/Desktop/jupyter/new_country_codes.csv"
df_countries = pd.read_csv(country_codes)
df_countries

,Unnamed: 0,StateAbb,CCode,StateNme,Names,SpeechCount
0,0,USA,2,United States of America,"United States,united states,USA,US,united stat...",NaN
1,1,CAN,20,Canada,"Canada,canada",NaN
2,2,BHM,31,Bahamas,"Bahamas,bahamas",NaN
3,3,CUB,40,Cuba,"Cuba,cuba",NaN
4,5,HAI,41,Haiti,"Haiti,haiti",NaN
5,7,DOM,42,Dominican Republic,"Dominican Republic,dominican republic,dominica...",NaN
6,9,JAM,51,Jamaica,"Jamaica,jamaica",NaN
7,10,TRI,52,Trinidad and Tobago,"Trinidad and Tobago,trinidad and tobago,trinid...",NaN
8,11,BAR,53,Barbados,"Barbados,barbados,barbado",NaN
9,12,DMA,54,Dominica,"Dominica,dominica,dominica is",NaN


In [12]:
#removes duplicates to validate data
df_countries = df_countries.drop_duplicates(subset=['StateNme'])
df_countries

,Unnamed: 0,StateAbb,CCode,StateNme,Names,SpeechCount
0,0,USA,2,United States of America,"United States,united states,USA,US,united stat...",NaN
1,1,CAN,20,Canada,"Canada,canada",NaN
2,2,BHM,31,Bahamas,"Bahamas,bahamas",NaN
3,3,CUB,40,Cuba,"Cuba,cuba",NaN
4,5,HAI,41,Haiti,"Haiti,haiti",NaN
5,7,DOM,42,Dominican Republic,"Dominican Republic,dominican republic,dominica...",NaN
6,9,JAM,51,Jamaica,"Jamaica,jamaica",NaN
7,10,TRI,52,Trinidad and Tobago,"Trinidad and Tobago,trinidad and tobago,trinid...",NaN
8,11,BAR,53,Barbados,"Barbados,barbados,barbado",NaN
9,12,DMA,54,Dominica,"Dominica,dominica,dominica is",NaN


In [13]:
def list_to_string(lst):
    s = ""
    for name in lst:
        s+= str(name) + ","
    s = s[:-1]
    return s

In [14]:
# add alternate names to list of country codes
def add_alternate_names():
    for index, row in df_countries.iterrows():
        for i in range(len(all_countries)):
            if row['StateNme'] in all_countries[i]:
                df_countries.loc[index, 'Names'] = list_to_string(all_countries[i])
# add_alternate_names()

In [15]:
#if there is no alternate name, add original name in
#CHECKS FOR NaN
for index, row in df_countries.iterrows():
    if isinstance(row['Names'],float):
        df_countries.loc[index, 'Names'] = str(row['StateNme'])
df_countries

,Unnamed: 0,StateAbb,CCode,StateNme,Names,SpeechCount
0,0,USA,2,United States of America,"United States,united states,USA,US,united stat...",NaN
1,1,CAN,20,Canada,"Canada,canada",NaN
2,2,BHM,31,Bahamas,"Bahamas,bahamas",NaN
3,3,CUB,40,Cuba,"Cuba,cuba",NaN
4,5,HAI,41,Haiti,"Haiti,haiti",NaN
5,7,DOM,42,Dominican Republic,"Dominican Republic,dominican republic,dominica...",NaN
6,9,JAM,51,Jamaica,"Jamaica,jamaica",NaN
7,10,TRI,52,Trinidad and Tobago,"Trinidad and Tobago,trinidad and tobago,trinid...",NaN
8,11,BAR,53,Barbados,"Barbados,barbados,barbado",NaN
9,12,DMA,54,Dominica,"Dominica,dominica,dominica is",NaN


In [16]:
#get all_ccodes for columns for final df
all_ccodes = []
for index, row in df_countries.iterrows():
    ccode = df_countries.loc[index, 'CCode']
    all_ccodes.append(ccode)

len(all_ccodes) #verifies all ccodes collected

223

# The following code analyzes the creates keywords
1. create dictionary of key words to flag


In [17]:
df_all_index = []
for s_index, s_row in df.iterrows():
    speech_id = df.loc[s_index, 'speech_id']
    df_all_index.append(speech_id)

len(df_all_index) #verifies all speeches accounted for

6252

In [18]:
# create empty df with ccode as columns and speech_id as rows
df_final = pd.DataFrame(columns=all_ccodes, index=df_all_index)
df_final

,2,20,31,40,41,42,51,52,53,54,...,983,986,987,990,1000,1001,1002,1003,1004,1005
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
#create dictionary of countries from speeches
def incr_country_dict(d, ID): 
    if ID not in d:
        d[ID] = 1
    else:
        d[ID] += 1
    return d

In [20]:
def country_count_by_speech(speech_id):
    #creates empty dict of all ccodes
    country_dict = {key: 0 for key in all_ccodes} 
    
    # uses SQL to grab content where speech_ids match
    speech = engine.execute("SELECT content FROM speeches WHERE speech_id = '"+ str(speech_id) +"'").fetchall().pop()[0]
    
    #itterating through each country/aliases of country name
    for c_index, c_row in df_countries.iterrows():
        country_names = df_countries.loc[c_index, 'Names']
        country_id = df_countries.loc[c_index, 'CCode']
        for country in country_names.split(','):
            if speech.find(country) != -1: #finding country in speech
                incr_country_dict(country_dict, country_id)
    return country_dict

In [21]:
# fill each row of df_final with country count
for speech_id, ccode in df_final.iterrows():
    df_final.loc[speech_id] = pd.Series(country_count_by_speech(speech_id))

In [22]:
df_final

,2,20,31,40,41,42,51,52,53,54,...,983,986,987,990,1000,1001,1002,1003,1004,1005
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
13,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25,1,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [23]:
#writing the edited df above into a csv
country_count_by_speech_path = "/Users/Andey/Desktop/jupyter/country_count_by_speech.csv"
write_csv(df_final, country_count_by_speech_path)